In [1]:
import pandas as pd

In [13]:
# data from the covid_dataset
df_hopkins = pd.read_csv('data/confirmed.csv')
# data for population of the worldbank
# skip non csv stuff in the first 4 rows
df_world_bank = pd.read_csv('data/wpp_population.csv', skiprows=4)

In [14]:
# take a look at the data
df_world_bank.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN


In [27]:
# list with columns which contain population data
df_world_bank_pop_columns = df_world_bank.columns[4:]

In [5]:
hk_countries = set(df_hopkins['Country/Region'].unique())
wb_countries = set(df_world_bank['Country Name'].unique())

In [6]:
# see what countries are missing
hk_countries - wb_countries

{'Bahamas',
 'Brunei',
 'Burma',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Czechia',
 'Diamond Princess',
 'Egypt',
 'Gambia',
 'Holy See',
 'Iran',
 'Korea, South',
 'Kyrgyzstan',
 'Laos',
 'MS Zaandam',
 'Russia',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Slovakia',
 'Syria',
 'Taiwan*',
 'US',
 'Venezuela',
 'Western Sahara',
 'Yemen'}

In [11]:
# get missing data from wikipedia
pop_other_sources = { 'Diamond Princess' : 3600, 
                        'Holy See' : 825, 
                        'Taiwan*' : 23_780_000 ,
                        'Western Sahara' : 595_060,
                        'MS Zaandam' :  (1243 + 586) }

In [10]:
pop_other_sources

{'Diamond Princess': 3600,
 'Holy See': 825,
 'Taiwan*': 23780000,
 'Western Sahara': 595060,
 'MS Zaandam': 1829}

In [16]:
hopkins_2_worldbank = { 'Bahamas': 'Bahamas, The', 
                 'Brunei' :  'Brunei Darussalam',  
                 'Burma' : 'Myanmar' ,  
                 'Congo (Brazzaville)': 'Congo, Dem. Rep.' ,
                 'Congo (Kinshasa)':  'Congo, Rep.',
                 'Czechia' :  'Czech Republic', 
                 'Egypt' : 'Egypt, Arab Rep.',
                 'Gambia' :  'Gambia, The',
                 'Iran' : 'Iran, Islamic Rep.',
                 'Korea, South' :'Korea, Rep.',
                 'Kyrgyzstan' : 'Kyrgyz Republic',
                 'Laos' : 'Lao PDR',
                 'Russia' : 'Russian Federation',
                 'Saint Kitts and Nevis' : 'St. Kitts and Nevis',
                 'Saint Lucia' : 'St. Lucia',
                 'Saint Vincent and the Grenadines' :  'St. Vincent and the Grenadines',
                 'Slovakia': 'Slovak Republic',
                 'Syria' :  'Syrian Arab Republic',
                 'US' : 'United States',
                 'Venezuela' :  'Venezuela, RB',
                 'Yemen' : 'Yemen, Rep.'
                } 

In [17]:
worldbank_2_hopkins = {v: k for k, v in hopkins_2_worldbank.items()}

In [19]:
# create new stripped down dataframe which onyl contains the country nam
df_pop = pd.DataFrame(columns=['Country/Region', 'Population'])

In [56]:
for c in wb_countries:
    pop_list = df_world_bank[df_world_bank['Country Name'] == c][df_world_bank_pop_columns].dropna(axis = 1).values
    # pop_list is array([]) with pop_list[-1] we get the inner list
    if len(pop_list[-1] > 0):
        if c in worldbank_2_hopkins:
            c = worldbank_2_hopkins[c]
        df_pop.loc[c,'Country/Region'] = c
        df_pop.loc[c,'Population'] = pop_list[-1][-1]

In [59]:
for k, v in pop_other_sources.items():
    df_pop.loc[k,'Country/Region'] = k
    df_pop.loc[k,'Population'] = v
    

In [61]:
df_pop.head()

,Country/Region,Population
Italy,Italy,6.04218e+07
Portugal,Portugal,1.02838e+07
World,World,7.59427e+09
Rwanda,Rwanda,1.23019e+07
Bulgaria,Bulgaria,7.02504e+06


In [62]:
df_pop.to_csv('data/population.csv')